In [1]:
import sys, os, json
import re, nltk
from nltk import pos_tag
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn, gensim
from sklearn.decomposition import PCA
import itertools
from gensim.corpora import Dictionary
from collections import defaultdict
sys.path.append(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/src/')
sys.path.append(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/src/')
os.chdir(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/data/')
from utils import preprocess_for_bow, experiment_result
from models.lda import LDAwrappers
from models.hdp import HDPwrapper
from models.gsdmm import MovieGroupProcessWrapper
from models.lftm import LFTMwrapper

In [2]:
with open('experiments_dataset1.json', 'r') as fout:
    res=json.load(fout)

In [3]:
res.keys()

dict_keys(['lda_experiment', 'gsdmm_experiment'])

In [135]:
with open('experiments_dataset2.json', 'r') as fout:
    res2=json.load(fout)

In [27]:
import pandas as pd
def to_df(res):
    params=defaultdict(list)
    for exp in res.keys():
        params['c_we'].append(res[exp]['coherence_metrics']['c_we']['c_we'])
        params['c_we_std'].append(res[exp]['coherence_metrics']['c_we']['c_we_std'])
        #params['num_topics'].append(res[exp]['number_topics'])
        for p in res[exp]['hyperparameters'].keys():
            params[p].append(res[exp]['hyperparameters'][p])
    #if params['num_topics'][0]==None:
    #    del params['num_topics']
    return pd.DataFrame(params)
    #return params


## LDA: word embedding coherence

#### DATASET 1

In [45]:
lda=to_df(res['lda_experiment'])
lda.iloc[lda.nlargest(5, 'c_we').index]

,c_we,c_we_std,num_topics,decay,passes
48,0.242069,0.087584,16,0.75,1
27,0.241629,0.049041,12,0.50,1
0,0.241507,0.046055,6,0.50,1
14,0.239710,0.087527,8,0.75,5
20,0.236841,0.057334,10,0.50,5


In [49]:
lda.iloc[lda.nsmallest(5, 'c_we_std').index]

,c_we,c_we_std,num_topics,decay,passes
6,0.213938,0.028108,6,0.90,1
18,0.199861,0.036153,10,0.50,1
50,0.216734,0.040622,16,0.75,5
8,0.208430,0.041338,6,0.90,5
39,0.205115,0.043111,14,0.75,1


In [36]:
import statsmodels.api as sm
y = lda['c_we']
x=lda.drop(columns=['c_we', 'c_we_std'])
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   c_we   R-squared:                       0.044
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                    0.8945
Date:                Thu, 02 Nov 2023   Prob (F-statistic):              0.449
Time:                        09:01:50   Log-Likelihood:                 188.76
No. Observations:                  63   AIC:                            -369.5
Df Residuals:                      59   BIC:                            -361.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2229      0.009     25.301      0.0

In [37]:
y = lda['c_we_std']
x=lda.drop(columns=['c_we', 'c_we_std'])
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:               c_we_std   R-squared:                       0.034
Model:                            OLS   Adj. R-squared:                 -0.015
Method:                 Least Squares   F-statistic:                    0.7010
Date:                Thu, 02 Nov 2023   Prob (F-statistic):              0.555
Time:                        09:02:35   Log-Likelihood:                 190.11
No. Observations:                  63   AIC:                            -372.2
Df Residuals:                      59   BIC:                            -363.7
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0561      0.009      6.502      0.0

#### DATASET 2


In [123]:
with open('res.txt', 'w') as fout:
    for t in res2['lda_experiment']['exp_35']['word_topic_pvalues']:
        fout.write('\n')
        for w in t['words']:
            fout.write(w+'\n')

In [126]:
res2['lda_experiment']

{'exp_0': {'number_topics': 6,
  'hyperparameters': {'num_topics': 6, 'decay': 0.5, 'passes': 1},
  'doc_topic_pvalues': {'0.35': 18,
   '0.50': 18,
   '0.60': 18,
   '0.75': 16,
   '0.90': 13,
   '0.95': 11},
  'word_topic_pvalues': [{'words': ['u093e',
     'u0947',
     'say',
     'u0930',
     'post',
     'check',
     'u094d',
     'u0915',
     'claim',
     'fact'],
    'weights': [0.01025896705687046,
     0.007516743615269661,
     0.007203998509794474,
     0.006842554081231356,
     0.006281844340264797,
     0.00611408706754446,
     0.005763133056461811,
     0.005627576727420092,
     0.005607025232166052,
     0.00529677327722311]},
   {'words': ['u093e',
     'claim',
     'video',
     'u0915',
     'u0947',
     'post',
     'u0930',
     'news',
     'fact',
     'use'],
    'weights': [0.00877037737518549,
     0.008653886616230011,
     0.008486028760671616,
     0.008170302025973797,
     0.008161536417901516,
     0.007215381134301424,
     0.005183343775570393

In [59]:
lda2=to_df(res2['lda_experiment'])
lda2.iloc[lda2.nlargest(5, 'c_we').index]

,c_we,c_we_std,num_topics,decay,passes
35,0.289717,0.109726,12,0.90,5
8,0.282400,0.152283,6,0.90,5
6,0.278735,0.126280,6,0.90,1
11,0.276158,0.139651,8,0.50,5
48,0.264954,0.146506,16,0.75,1


In [60]:
lda2.iloc[lda2.nsmallest(5, 'c_we_std').index]

,c_we,c_we_std,num_topics,decay,passes
44,0.239121,0.084175,14,0.90,5
16,0.224375,0.092427,8,0.90,2
22,0.225395,0.093985,10,0.75,2
14,0.257997,0.099312,8,0.75,5
24,0.259601,0.099661,10,0.90,1


In [61]:
y = lda2['c_we']
x=lda2.drop(columns=['c_we', 'c_we_std'])
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   c_we   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                 -0.027
Method:                 Least Squares   F-statistic:                    0.4474
Date:                Thu, 02 Nov 2023   Prob (F-statistic):              0.720
Time:                        10:55:27   Log-Likelihood:                 141.55
No. Observations:                  63   AIC:                            -275.1
Df Residuals:                      59   BIC:                            -266.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2358      0.019     12.646      0.0

In [62]:
y = lda2['c_we_std']
x=lda2.drop(columns=['c_we', 'c_we_std'])
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:               c_we_std   R-squared:                       0.143
Model:                            OLS   Adj. R-squared:                  0.099
Method:                 Least Squares   F-statistic:                     3.268
Date:                Thu, 02 Nov 2023   Prob (F-statistic):             0.0274
Time:                        10:55:40   Log-Likelihood:                 179.37
No. Observations:                  63   AIC:                            -350.7
Df Residuals:                      59   BIC:                            -342.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1204      0.010     11.773      0.0

## LDA: UMASS coherence

#### dataset 1


In [83]:
def to_df2(res):
    params=defaultdict(list)
    for exp in res.keys():
        params['u_mass'].append(res[exp]['coherence_metrics']['u_mass']['u_mass'])
        params['u_mass_std'].append(res[exp]['coherence_metrics']['u_mass']['u_mass_std'])
        #params['num_topics'].append(res[exp]['number_topics'])
        for p in res[exp]['hyperparameters'].keys():
            params[p].append(res[exp]['hyperparameters'][p])
    #if params['num_topics'][0]==None:
    #    del params['num_topics']
    return pd.DataFrame(params)

In [84]:
lda=to_df2(res['lda_experiment'])
lda.iloc[lda.nlargest(5, 'u_mass').index]

,u_mass,u_mass_std,num_topics,decay,passes
1,-2.623792,1.004971,6,0.50,2
17,-2.811936,0.944704,8,0.90,5
21,-2.813197,0.818438,10,0.75,1
13,-2.854348,0.983164,8,0.75,2
33,-2.876633,0.755457,12,0.90,1


In [85]:
lda.iloc[lda.nsmallest(5, 'u_mass_std').index]

,u_mass,u_mass_std,num_topics,decay,passes
16,-3.062849,0.168509,8,0.90,2
27,-3.248157,0.193150,12,0.50,1
6,-3.159944,0.197357,6,0.90,1
34,-3.214630,0.217121,12,0.90,2
4,-3.194717,0.230034,6,0.75,2


In [87]:
y = lda['u_mass']
x=lda.drop(columns=['u_mass', 'u_mass_std'])
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 u_mass   R-squared:                       0.053
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     1.105
Date:                Thu, 02 Nov 2023   Prob (F-statistic):              0.354
Time:                        11:55:54   Log-Likelihood:                 17.807
No. Observations:                  63   AIC:                            -27.61
Df Residuals:                      59   BIC:                            -19.04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.1858      0.133    -23.970      0.0

In [88]:
y = lda['u_mass_std']
x=lda.drop(columns=['u_mass', 'u_mass_std'])
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             u_mass_std   R-squared:                       0.133
Model:                            OLS   Adj. R-squared:                  0.088
Method:                 Least Squares   F-statistic:                     3.006
Date:                Thu, 02 Nov 2023   Prob (F-statistic):             0.0374
Time:                        11:56:18   Log-Likelihood:                0.23039
No. Observations:                  63   AIC:                             7.539
Df Residuals:                      59   BIC:                             16.11
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5936      0.176      3.379      0.0

#### dataset 2

In [147]:
lda2=to_df2(res2['lda_experiment'])
lda2.iloc[lda2.nlargest(5, 'u_mass').index]

,u_mass,u_mass_std,num_topics,decay,passes
25,-0.461509,0.377412,10,0.90,2
4,-0.554065,0.100125,6,0.75,2
3,-0.570491,0.343090,6,0.75,1
17,-0.587406,0.492029,8,0.90,5
0,-0.594014,0.434840,6,0.50,1


In [156]:
res2['lda_experiment']['exp_4']['word_topic_pvalues'][3]['words']

['claim',
 'covid',
 'fact',
 'say',
 'video',
 'check',
 'use',
 'share',
 'news',
 'story']

In [148]:
lda2.iloc[lda2.nsmallest(5, 'u_mass_std').index]

,u_mass,u_mass_std,num_topics,decay,passes
4,-0.554065,0.100125,6,0.75,2
15,-0.653324,0.316765,8,0.90,1
3,-0.570491,0.343090,6,0.75,1
46,-0.713196,0.352016,16,0.50,2
25,-0.461509,0.377412,10,0.90,2


In [149]:
y = lda2['u_mass']
x=lda2.drop(columns=['u_mass', 'u_mass_std'])
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 u_mass   R-squared:                       0.061
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     1.274
Date:                Fri, 03 Nov 2023   Prob (F-statistic):              0.292
Time:                        09:46:37   Log-Likelihood:                -12.349
No. Observations:                  63   AIC:                             32.70
Df Residuals:                      59   BIC:                             41.27
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.8466      0.215     -3.947      0.0

In [112]:
y = lda2['u_mass_std']
x=lda2.drop(columns=['u_mass', 'u_mass_std'])
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             u_mass_std   R-squared:                       0.073
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     1.553
Date:                Thu, 02 Nov 2023   Prob (F-statistic):              0.210
Time:                        15:11:34   Log-Likelihood:                -54.773
No. Observations:                  63   AIC:                             117.5
Df Residuals:                      59   BIC:                             126.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7614      0.421      1.810      0.0

## GSDMM: word embedding coherence

#### dataset 1

In [139]:
gsdmm=to_df(res['gsdmm_experiment'])
gsdmm.iloc[gsdmm.nlargest(5, 'c_we').index]

,c_we,c_we_std,K,alpha,beta,n_iters
1,0.365417,0.0,6,0.1,0.1,5
34,0.365417,0.0,12,0.2,0.2,5
2,0.352392,0.0,6,0.1,0.1,15
13,0.332914,0.0,9,0.1,0.1,5
61,0.322158,0.0,21,0.1,0.1,5


In [115]:
gsdmm.iloc[gsdmm.nsmallest(5, 'c_we_std').index]

,c_we,c_we_std,K,alpha,beta,n_iters
0,0.184020,0.0,6,0.1,0.1,1
1,0.365417,0.0,6,0.1,0.1,5
2,0.352392,0.0,6,0.1,0.1,15
3,0.183047,0.0,6,0.1,0.2,1
4,0.240072,0.0,6,0.1,0.2,5


In [117]:
y = gsdmm['c_we']
x=gsdmm.drop(columns=['c_we', 'c_we_std'])
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   c_we   R-squared:                       0.103
Model:                            OLS   Adj. R-squared:                  0.058
Method:                 Least Squares   F-statistic:                     2.273
Date:                Thu, 02 Nov 2023   Prob (F-statistic):             0.0687
Time:                        15:14:52   Log-Likelihood:                 118.37
No. Observations:                  84   AIC:                            -226.7
Df Residuals:                      79   BIC:                            -214.6
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3047      0.034      8.869      0.0

#### dataset 2

In [136]:
gsdmm=to_df2(res2['gsdmm_experiment'])
gsdmm.iloc[gsdmm.nlargest(5, 'u_mass').index]

,u_mass,u_mass_std,K,alpha,beta,n_iters
4,-0.509983,0.054108,6,0.1,0.2,5
43,-0.636831,0.184751,15,0.2,0.1,5
26,-0.683594,0.249467,12,0.1,0.1,15
50,-0.687103,0.254352,18,0.1,0.1,15
65,-0.733169,0.318711,21,0.1,0.2,15


In [137]:
gsdmm.iloc[gsdmm.nsmallest(5, 'u_mass_std').index]

,u_mass,u_mass_std,K,alpha,beta,n_iters
4,-0.509983,0.054108,6,0.1,0.2,5
43,-0.636831,0.184751,15,0.2,0.1,5
26,-0.683594,0.249467,12,0.1,0.1,15
50,-0.687103,0.254352,18,0.1,0.1,15
65,-0.733169,0.318711,21,0.1,0.2,15


In [138]:
y = gsdmm['u_mass']
x=gsdmm.drop(columns=['u_mass', 'u_mass_std'])
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 u_mass   R-squared:                       0.334
Model:                            OLS   Adj. R-squared:                  0.301
Method:                 Least Squares   F-statistic:                     9.916
Date:                Fri, 03 Nov 2023   Prob (F-statistic):           1.49e-06
Time:                        08:27:35   Log-Likelihood:                -18.533
No. Observations:                  84   AIC:                             47.07
Df Residuals:                      79   BIC:                             59.22
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.0336      0.175     -5.897      0.0

## GSDMM: UMASS coherence

#### dataset 1

In [145]:
gsdmm=to_df2(res['gsdmm_experiment'])
gsdmm.iloc[gsdmm.nlargest(5, 'u_mass').index]

,u_mass,u_mass_std,K,alpha,beta,n_iters
2,-2.448810,0.978883,6,0.1,0.1,15
1,-2.456073,0.978553,6,0.1,0.1,5
10,-2.472700,1.012691,6,0.2,0.2,5
8,-2.474787,0.989685,6,0.2,0.1,15
0,-2.549945,0.977607,6,0.1,0.1,1


In [146]:
y = gsdmm['u_mass_std']
x=gsdmm.drop(columns=['u_mass', 'u_mass_std'])
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             u_mass_std   R-squared:                       0.528
Model:                            OLS   Adj. R-squared:                  0.504
Method:                 Least Squares   F-statistic:                     22.12
Date:                Fri, 03 Nov 2023   Prob (F-statistic):           2.81e-12
Time:                        09:09:27   Log-Likelihood:                -90.780
No. Observations:                  84   AIC:                             191.6
Df Residuals:                      79   BIC:                             203.7
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.5306      0.414     -1.281      0.2

#### dataset 2

In [143]:
gsdmm=to_df2(res2['gsdmm_experiment'])
gsdmm.iloc[gsdmm.nlargest(5, 'u_mass').index]

,u_mass,u_mass_std,K,alpha,beta,n_iters
4,-0.509983,0.054108,6,0.1,0.2,5
43,-0.636831,0.184751,15,0.2,0.1,5
26,-0.683594,0.249467,12,0.1,0.1,15
50,-0.687103,0.254352,18,0.1,0.1,15
65,-0.733169,0.318711,21,0.1,0.2,15


In [144]:
y = gsdmm['u_mass_std']
x=gsdmm.drop(columns=['u_mass', 'u_mass_std'])
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             u_mass_std   R-squared:                       0.121
Model:                            OLS   Adj. R-squared:                  0.077
Method:                 Least Squares   F-statistic:                     2.728
Date:                Fri, 03 Nov 2023   Prob (F-statistic):             0.0349
Time:                        08:57:34   Log-Likelihood:                -8.3064
No. Observations:                  84   AIC:                             26.61
Df Residuals:                      79   BIC:                             38.77
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5481      0.155      3.532      0.0

### LDA: cluster title generation

In [97]:
from transformers import pipeline
from bertopic.representation import TextGeneration

def prompt(topicwords):
    prompt= f"I have a topic described by the following keywords: [{', '.join(map(str, topicwords))}]. Based on the previous keywords, what is this topic about?"
    return prompt

# Create your representation model
generator = pipeline('text2text-generation', model='google/flan-t5-base')
representation_model = TextGeneration(generator)